In [1]:
import os
import requests
import pandas as pd
import io
import urllib.request

os.chdir("D:\Stocks\PersianCal")
import PersianCal
import numpy as np

In [2]:
os.chdir('D:/Stocks/')
file = 'Codes.csv'
codes = pd.read_csv(file)
# code = str(codes.Codes[code_number])

In [3]:
code_number = 100

# code = str(codes.Codes[code_number])

code = '49776615757150035'

url = "http://www.tsetmc.com/tsev2/data/Export-txt.aspx?t=i&a=1&b=0&i=" + code
r = requests.get(url, allow_redirects=True).content

# open('7769065543658313.csv', 'wb').write(r.content);

In [4]:
df = pd.read_csv(io.StringIO(r.decode('utf-8')))

In [5]:
## Datetime conversion
df['<DTYYYYMMDD>'] = pd.to_datetime(df['<DTYYYYMMDD>'], format='%Y%m%d')

# Dropping any NaNs
df.dropna(inplace=True)

df.rename(columns={"<DTYYYYMMDD>": "Date"}, inplace=True)

In [6]:
df['Date'] =pd.to_datetime(df.Date)
df = df.sort_values(by=['Date'])

In [7]:
d = []
for i in range(len(df["Date"])):
    d.append(PersianCal.Gregorian(str(df["Date"][i])[0:10]).persian_string()) #Tabdil tarikh be Irani

In [8]:
for i in range(len(df["Date"])):
    df["Date"] = df["Date"].replace(df["Date"][i], d[i])

In [9]:
date_list = []
for date in df["Date"]:
    date_list.append(date[:7])
    
Dates = df["Date"].tolist() #kole tarikh ha
myDates = list(dict.fromkeys(date_list)) #Eshterake Tatikh ha

In [10]:
ext = np.zeros((len(myDates), 2)) #[[max], [min]]
ind = np.zeros((len(myDates), 1)).tolist()

#first indicator
for date in Dates:
    if (date[:7] in myDates[0]):
        ind1 = Dates.index(date) #یافتن مکان آخرین روز ماه

ind[0] = ind1

a = [[], [], [], []]
names = ["<FIRST>", "<HIGH>", "<LOW>", "<CLOSE>"]
for i in range(4):
    for j in names:
        a[i] = df[j][0:ind1].tolist()
        
ext[0][0] = max(max(a))
ext[0][1] = min(min(a))

In [11]:
for i in range(1, len(myDates)):
    for date in Dates:
        if (date[:7] in myDates[i]):
            ind1 = Dates.index(date)
    ind[i] = ind1
    a = [[], [], [], []]
    for ii in range(4):
        for j in names:
            a[ii] = df[j][ind[i-1]:ind[i]].tolist()
    ext[i][0] = max(max(a))
    ext[i][1] = min(min(a))

In [12]:
Data = {"Date": myDates,
        "Max": ext[:, 0],
        "Min": ext[:, 1]}
DataF = pd.DataFrame(Data)

DataF.set_index('Date', inplace=True)

In [13]:
os.chdir("D:/")

file_name = str(df['<TICKER>'][1]).replace('*', '')

DataF.to_csv(file_name + '.csv')

In [14]:
# import pylab as pp
# %matplotlib notebook
# pp.plot(DataF["Date"], DataF["Max"], color='skyblue', label = "Max")
# pp.plot(DataF["Date"], DataF["Min"], color='red', label = "Min")
# pp.legend();